In [1]:
import pandas as pd
import os
import requests
from PIL import Image
import warnings
import shutil


warnings.filterwarnings("ignore")

In [2]:
#read full training set csv, assign column names and index, save for future. Do not run this cell again
#df3 = pd.read_csv(r'large_meta_data/Image_Labels_Subset_Train_GCC-Labels-training.tsv', sep='\t', header=None, names=['caption', 'link', 'objects','mid', 'object_confidences'])
#print(len(df3))
# #df3.to_csv(r'large_meta_data/Image_Labels_Object_Subset.tsv',index_label='index')

In [3]:
train_gcc_meta = pd.read_csv(r'large_meta_data/Image_Labels_Object_Subset.tsv')
print(len(train_gcc_meta))
train_gcc_meta.head(2)

2007528


,index,caption,link,objects,mid,object_confidences
0,0,christmas tree on a black background .,https://thumb1.shutterstock.com/display_pic_wi...,"christmas tree,christmas decoration,font,text,...","/m/025nd,/m/05fc9mj,/m/03gq5hm,/m/07s6nbt,/m/0...","0.9818305373191833,0.952756941318512,0.9227379..."
1,1,item : drawing of a figure surrounded by person,https://i.pinimg.com/736x/f9/fd/48/f9fd4878090...,"drawing,modern art,line,visual arts,art,sketch...","/m/02csf,/m/015r61,/m/03scnj,/m/0p9xx,/m/0jjw,...","0.8945257067680359,0.8489813804626465,0.828646..."


In [5]:
#save the current working directory to use later
cwd = os.getcwd()
cwd

'C:\\Users\\apra\\Desktop\\SPRING 2021\\DS 4400\\ML1_Final_Proj'

In [4]:
#os.chdir('..')
#os.chdir('ML1_Final_Proj/')

In [53]:
#we start a dataframe with indices of already sampled data to keep track of the data we already have. This does not include includeabs
#initial 10k samples since the indices were lost.Do not run this cell again

initial_sample = train_gcc_meta.sample(1)
if(not os.path.exists(r'data/small_sample')):
    os.mkdir(r'data/small_sample')
    os.mkdir(r'data/small_sample/images')
os.chdir(r'data/small_sample/images')

#keep track of number of samples sucessfully taken
num_sampled = 0

successfully_sampled_indices = []
dead_link_indices = []

#faster to append to directory list than directly dataframe
cleaned_sample = []

for idx, row in initial_sample.iterrows():
    url = row['link']
    try:
        r = requests.get(url, verify=False, timeout=2, stream=True)
        if r.status_code == 200:
            #append the row to the meta_dataframe for this sample
            print(row)
            cleaned_sample.append(row.to_dict())
            with open(str(idx)+'.png', 'wb') as out_file:
                #download the image and name it with its index in train.gcc
                shutil.copyfileobj(r.raw, out_file)
                #store the successfully sampled image's index
                successfully_sampled_indices.append(idx)
                print(idx)
                num_sampled +=1
        else:
            print(f"failed to download image: {idx}")
            dead_link_indices.append(idx)
            
    except requests.exceptions.ReadTimeout:
        print (f"Read timed out for index: {idx}")
    except Exception as e:
        print(e)
        print (f"Other requests excpetion for index: {idx}")

#save the dataframes

#samples metadata in parent
os.chdir('..')
sample_meta_df = pd.DataFrame.from_dict(cleaned_sample)
sample_meta_df = sample_meta_df.rename(columns = {'index':'global_index'})
sample_meta_df.to_csv('meta_data', index = False)



#global sampled indices and dead links dataframe in project meta_data folder
os.chdir(cwd)
os.chdir(r'meta_data')
successfully_sampled_indices_df = pd.Series(successfully_sampled_indices)
successfully_sampled_indices_df.to_csv('sampled_idx_list',index = False, header = ['index'])

dead_link_indice_df = pd.Series(dead_link_indices)
dead_link_indice_df.to_csv('dead_link_idx_list', index=False, header = ['index'])


#go back to cwd
os.chdir(cwd)



        





index                                                            121763
caption                                   actor arrives at the premiere
link                  http://media.gettyimages.com/photos/actor-dams...
objects               forehead,premiere,smile,fashion design,caesar cut
mid                   /m/025kyy,/m/03n3f3,/m/019nj4,/m/02qqd4n,/m/06...
object_confidences    0.8217551112174988,0.6771990656852722,0.641064...
Name: 121763, dtype: object
121763


In [12]:
def sample_n_new_images(n, to_dirname, meta_df, success_file_path, dead_file_path):
    """
    Samples and downloads n previously unsampled images from links in meta_df and saves them and their meta_data to a directory 
    with name to_dirname .Uses and updates successfully sampled indices and dead link indices in success_file_path and dead_file_path.
    
    """
    cwd = os.getcwd()
    #make directories to save sampled data
    to_dir = os.path.join(r'data',to_dirname)
    if(not os.path.exists(to_dir)):
        os.mkdir(to_dir)
    to_dir_images = os.path.join(to_dir,'images')
    if(not os.path.exists(to_dir_images)):
        os.mkdir(to_dir_images)
    

    
    successfully_sampled_indices = pd.read_csv(success_file_path, header = 0)
    dead_link_indices = pd.read_csv(dead_file_path, header = 0)

    
    #filter full dataframe to exclude above indices
    
    new_links = meta_df.drop(labels = successfully_sampled_indices['index'], axis='index')
    new_alive_links = new_links.drop(labels =  dead_link_indices['index'], axis = 'index')
    
    
    #randomly shuffle the filtered datframe
    num_good_rows = len(new_alive_links)
    n_sample = new_alive_links.sample(n=num_good_rows)
    
    num_successfully_sampled = 0
    #to update the indice lists for successfull and failed
    success_indices = successfully_sampled_indices['index'].tolist()
    dead_indices = dead_link_indices['index'].tolist()
    
    
    #sample metadata
    cleaned_sample = []
    
    #change directory to images directory
    os.chdir(to_dir_images)
    for idx, row in n_sample.iterrows():
        #break loop if we hit the number we want
        if(num_successfully_sampled == n):
            break
        url = row['link']
        try:
            r = requests.get(url, verify=False, timeout=2, stream=True)
            if r.status_code == 200:               
                file_name = str(idx)+'.png'
                with open(file_name, 'wb') as out_file:
                    #download the image and name it with its index in train.gcc
                    shutil.copyfileobj(r.raw, out_file)
                    
                img = Image.open(file_name) # open the image file
                img.verify() # verify that it is, in fact an image    
                #store the successfully sampled image's index
                success_indices.append(idx)
                num_successfully_sampled +=1
                #append the row to the meta_dataframe for this sample
                cleaned_sample.append(row.to_dict())
            else:
                print(f"failed to download image: {idx}")
                dead_indices.append(idx)

        except requests.exceptions.ReadTimeout:
            print (f"Read timed out for index: {idx}")
            
        except Exception as e:
            print(e)
            print (f"Other requests excpetion for index: {idx}")
            if(os.path.exists(file_name)):
                os.remove(file_name)
        except (IOError, SyntaxError) as e:
            print('Bad file:', idx) # print out the names of corrupt files
            if(os.path.exists(file_name)):
                os.remove(file_name)
        # except error as e:
        #     if(os.path.exists(file_name)):
        #         os.remove(file_name)
            

    
    
    #add newly successful and failed links to respectful dataframes
    
            
    print(f"successfully sampled {num_successfully_sampled} new indices")
    
    #save the dataframes

    #samples metadata in parent
    
    sample_meta_df = pd.DataFrame.from_dict(cleaned_sample)
    sample_meta_df.rename(columns = {'index':'global_index'}, inplace = True)
    
    #to be safe, we remove all rows for which a file was not saved for whatever reason 
    image_names = os.listdir('.')
    image_names = [i.split('.')[0] for i in image_names]

    
    for idx,file in sample_meta_df.iterrows():
        global_index = file['global_index']
        if not (str(global_index) in image_names):
            sample_meta_df.drop(sample_meta_df[sample_meta_df['global_index']==global_index].index, axis='index', inplace=True)
            num_successfully_sampled -= 1
            dead_indices.append(global_index)
            success_indices.remove(global_index)
            print("dropped", idx,global_index)
    
    
    os.chdir('..')
    sample_meta_df.to_csv('meta_data', index = False)



    #global sampled indices and dead links dataframe in project meta_data folder
    os.chdir(cwd)
    os.chdir(r'meta_data')
    
    
    
    
    success_path = os.path.basename(success_file_path)
    successfully_sampled_indices_df = pd.Series(success_indices)
    successfully_sampled_indices_df.to_csv(success_path,index = False, header = ['index'])

    dead_path = os.path.basename(dead_file_path)
    dead_link_indice_df = pd.Series(dead_indices)
    dead_link_indice_df.to_csv(dead_path, index=False, header = ['index'])


    #go back to cwd
    os.chdir(cwd)
    
        
        
    
    
    
    
    
    
    
    
    
    
    
    

In [13]:
#sample_n_new_images(20000, r'20k_sample_2', train_gcc_meta, r'meta_data/dummy_success.csv', r'meta_data/dummy_fail.csv')

In [15]:
 sample_n_new_images(20000, r'20k_sample_2', train_gcc_meta, r'meta_data\sampled_idx_list', r'meta_data\dead_link_idx_list')

cannot identify image file '224422.png'
Other requests excpetion for index: 224422
cannot identify image file '328441.png'
Other requests excpetion for index: 328441
cannot identify image file '1740048.png'
Other requests excpetion for index: 1740048
failed to download image: 139396
failed to download image: 466175
failed to download image: 66599
cannot identify image file '1222321.png'
Other requests excpetion for index: 1222321
cannot identify image file '1842828.png'
Other requests excpetion for index: 1842828
failed to download image: 469920
failed to download image: 33877
failed to download image: 194501
failed to download image: 783273
failed to download image: 1664456
failed to download image: 1369349
failed to download image: 1613888
failed to download image: 563322
cannot identify image file '1374710.png'
Other requests excpetion for index: 1374710
cannot identify image file '627605.png'
Other requests excpetion for index: 627605
failed to download image: 1748986
cannot identi

failed to download image: 1847017
cannot identify image file '1309277.png'
Other requests excpetion for index: 1309277
failed to download image: 598975
cannot identify image file '291915.png'
Other requests excpetion for index: 291915
Read timed out for index: 340557
cannot identify image file '984843.png'
Other requests excpetion for index: 984843
failed to download image: 756462
failed to download image: 1349515
cannot identify image file '1010264.png'
Other requests excpetion for index: 1010264
Read timed out for index: 254354
cannot identify image file '1348255.png'
Other requests excpetion for index: 1348255
cannot identify image file '1410234.png'
Other requests excpetion for index: 1410234
cannot identify image file '847779.png'
Other requests excpetion for index: 847779
failed to download image: 208503
Read timed out for index: 615666
Read timed out for index: 140212
cannot identify image file '1730811.png'
Other requests excpetion for index: 1730811
Read timed out for index: 1

failed to download image: 1214818
failed to download image: 605871
failed to download image: 1563216
failed to download image: 157052
failed to download image: 1184375
cannot identify image file '719266.png'
Other requests excpetion for index: 719266
Read timed out for index: 1994627
failed to download image: 1674562
cannot identify image file '1171412.png'
Other requests excpetion for index: 1171412
Read timed out for index: 474167
cannot identify image file '77251.png'
Other requests excpetion for index: 77251
failed to download image: 1390928
cannot identify image file '463492.png'
Other requests excpetion for index: 463492
cannot identify image file '1391954.png'
Other requests excpetion for index: 1391954
failed to download image: 1794204
Read timed out for index: 1306939
cannot identify image file '683169.png'
Other requests excpetion for index: 683169
failed to download image: 1352757
cannot identify image file '466503.png'
Other requests excpetion for index: 466503
Read timed o

cannot identify image file '539844.png'
Other requests excpetion for index: 539844
cannot identify image file '1690306.png'
Other requests excpetion for index: 1690306
Read timed out for index: 553537
failed to download image: 1574682
failed to download image: 1643444
failed to download image: 1046225
cannot identify image file '1642006.png'
Other requests excpetion for index: 1642006
failed to download image: 1687151
failed to download image: 1315118
cannot identify image file '1657552.png'
Other requests excpetion for index: 1657552
failed to download image: 1370861
cannot identify image file '219779.png'
Other requests excpetion for index: 219779
cannot identify image file '1914482.png'
Other requests excpetion for index: 1914482
cannot identify image file '1412134.png'
Other requests excpetion for index: 1412134
cannot identify image file '1727375.png'
Other requests excpetion for index: 1727375
failed to download image: 580447
failed to download image: 1601142
cannot identify imag

failed to download image: 95750
cannot identify image file '1270620.png'
Other requests excpetion for index: 1270620
Read timed out for index: 663140
cannot identify image file '1784480.png'
Other requests excpetion for index: 1784480
Read timed out for index: 744445
cannot identify image file '149058.png'
Other requests excpetion for index: 149058
cannot identify image file '613051.png'
Other requests excpetion for index: 613051
cannot identify image file '1860548.png'
Other requests excpetion for index: 1860548
Read timed out for index: 98479
failed to download image: 502286
cannot identify image file '482751.png'
Other requests excpetion for index: 482751
cannot identify image file '1028571.png'
Other requests excpetion for index: 1028571
Read timed out for index: 1424226
failed to download image: 1265078
failed to download image: 1707994
Read timed out for index: 948679
failed to download image: 602634
failed to download image: 1197120
cannot identify image file '1589309.png'
Other

...

Read timed out for index: 1044781
failed to download image: 136523
failed to download image: 4212
cannot identify image file '1599779.png'
Other requests excpetion for index: 1599779
cannot identify image file '1761090.png'
Other requests excpetion for index: 1761090
Read timed out for index: 1458349
cannot identify image file '1331869.png'
Other requests excpetion for index: 1331869
failed to download image: 1451977
cannot identify image file '42172.png'
Other requests excpetion for index: 42172
failed to download image: 1353854
cannot identify image file '639107.png'
Other requests excpetion for index: 639107
cannot identify image file '1721570.png'
Other requests excpetion for index: 1721570
failed to download image: 1128121
failed to download image: 920029
cannot identify image file '211651.png'
Other requests excpetion for index: 211651
cannot identify image file '391575.png'
Other requests excpetion for index: 391575
cannot identify image file '1337176.png'
Other requests excpeti

cannot identify image file '946612.png'
Other requests excpetion for index: 946612
cannot identify image file '367908.png'
Other requests excpetion for index: 367908
cannot identify image file '941108.png'
Other requests excpetion for index: 941108
failed to download image: 488577
cannot identify image file '782772.png'
Other requests excpetion for index: 782772
failed to download image: 1057398
cannot identify image file '312680.png'
Other requests excpetion for index: 312680
cannot identify image file '1082392.png'
Other requests excpetion for index: 1082392
cannot identify image file '455350.png'
Other requests excpetion for index: 455350
cannot identify image file '1043804.png'
Other requests excpetion for index: 1043804
failed to download image: 549067
failed to download image: 621628
failed to download image: 1092559
cannot identify image file '181973.png'
Other requests excpetion for index: 181973
failed to download image: 1760609
failed to download image: 1447328
cannot identif

cannot identify image file '1292736.png'
Other requests excpetion for index: 1292736
cannot identify image file '693547.png'
Other requests excpetion for index: 693547
failed to download image: 1217349
failed to download image: 2000658
failed to download image: 235080
cannot identify image file '700908.png'
Other requests excpetion for index: 700908
failed to download image: 1557216
cannot identify image file '864158.png'
Other requests excpetion for index: 864158
cannot identify image file '395863.png'
Other requests excpetion for index: 395863
cannot identify image file '811953.png'
Other requests excpetion for index: 811953
failed to download image: 1972566
failed to download image: 1333212
failed to download image: 1689802
failed to download image: 157325
failed to download image: 1950606
failed to download image: 1457452
Read timed out for index: 1350121
cannot identify image file '1282202.png'
Other requests excpetion for index: 1282202
cannot identify image file '1498488.png'
Ot

failed to download image: 843458
Read timed out for index: 1170483
cannot identify image file '956208.png'
Other requests excpetion for index: 956208
failed to download image: 1144781
failed to download image: 923411
cannot identify image file '1451421.png'
Other requests excpetion for index: 1451421
failed to download image: 1740730
failed to download image: 1135570
cannot identify image file '1079585.png'
Other requests excpetion for index: 1079585
failed to download image: 111791
failed to download image: 1983433
failed to download image: 1130878
failed to download image: 1727437
cannot identify image file '282466.png'
Other requests excpetion for index: 282466
cannot identify image file '1733069.png'
Other requests excpetion for index: 1733069
cannot identify image file '1701468.png'
Other requests excpetion for index: 1701468
failed to download image: 1691752
failed to download image: 999945
Read timed out for index: 1791922
failed to download image: 1384835
cannot identify image 

cannot identify image file '53666.png'
Other requests excpetion for index: 53666
cannot identify image file '300762.png'
Other requests excpetion for index: 300762
Read timed out for index: 980168
cannot identify image file '2002745.png'
Other requests excpetion for index: 2002745
failed to download image: 404775
cannot identify image file '561914.png'
Other requests excpetion for index: 561914
failed to download image: 662670
failed to download image: 585235
failed to download image: 1636857
failed to download image: 1876625
failed to download image: 582734
cannot identify image file '1013675.png'
Other requests excpetion for index: 1013675
cannot identify image file '52758.png'
Other requests excpetion for index: 52758
cannot identify image file '961954.png'
Other requests excpetion for index: 961954
cannot identify image file '371237.png'
Other requests excpetion for index: 371237
failed to download image: 1671223
failed to download image: 626368
cannot identify image file '1875228.

In [90]:
# #now to verify the downloaded data and see if any were not downloaded properly

# def verify_downloads(down_dirname):
#     cwd = os.getcwd()
    
#    #find dirmname
#     down_dir = os.path.join(r'data',down_dirname)
#     down_dir_images = os.path.join(down_dir,'images')
#     down_dir_meta = os.path.join(down_dir, 'meta_data')
#     cleaned_meta_path = os.path.join(down_dir, 'cleaned_meta_data')
#     bad_files_path = os.path.join(down_dir, 'bad_image_indices')
   

#     sample_df = pd.read_csv(down_dir_meta, index_col='index')

#     good_count = 0
#     badFiles = []
#     badNumbers = []
    
#     print(len(sample_df))


#     for filename in os.listdir(down_dir_images):
#         if filename.endswith('.png'):
#             try:
#                 file_path = os.path.join(down_dir_images, filename)
#                 img = Image.open(file_path) # open the image file
#                 img.verify() # verify that it is, in fact an image
#                 good_count+=1
#             except (IOError, SyntaxError) as e:
#                 number = os.path.splitext(filename)[0]
#                 badNumbers.append(int(number))
#                 badFiles.append(filename)
#                 print('Bad file:', filename) # print out the names of corrupt files
                

  
#     for bad in badFiles:    
#         os.remove(os.path.join(down_dir_images,  bad))

    
#     cleaned_meta_df = sample_df.drop(labels = badNumbers, axis='index')
#     cleaned_meta_df.to_csv(cleaned_meta_path)
    
#     bad_indices_df = pd.Series(badNumbers)
#     bad_indices_df.to_csv(bad_files_path,index = False, header = ['index'])
    
#     print("Number of workable images: ", good_count)
    

20002
Bad file: 1006951.png
Bad file: 1007762.png
Bad file: 1010583.png
Bad file: 1012717.png
Bad file: 1013445.png
Bad file: 1018792.png
Bad file: 1022543.png
Bad file: 1024948.png
Bad file: 1029012.png
Bad file: 103089.png
Bad file: 1034828.png
Bad file: 1035169.png
Bad file: 1042929.png
Bad file: 1043065.png
Bad file: 1045449.png
Bad file: 1048066.png
Bad file: 1048214.png
Bad file: 105067.png
Bad file: 1053247.png
Bad file: 1055913.png
Bad file: 1058284.png
Bad file: 106226.png
Bad file: 1064937.png
Bad file: 1065785.png
Bad file: 1096882.png
Bad file: 1101031.png
Bad file: 1113691.png
Bad file: 1115702.png
Bad file: 1134195.png
Bad file: 1142352.png
Bad file: 1142386.png
Bad file: 1143249.png
Bad file: 1145298.png
Bad file: 1157305.png
Bad file: 1161021.png
Bad file: 1161151.png
Bad file: 1161292.png
Bad file: 1163998.png
Bad file: 1166245.png
Bad file: 1170375.png
Bad file: 1173401.png
Bad file: 1173627.png
Bad file: 1175244.png
Bad file: 1177796.png
Bad file: 1178884.png
Bad fil

Bad file: 245442.png
Bad file: 2460742.png
Bad file: 2467758.png
Bad file: 246982.png
Bad file: 2471230.png
Bad file: 2473687.png
Bad file: 2479699.png
Bad file: 2483970.png
Bad file: 2486319.png
Bad file: 2489413.png
Bad file: 2491707.png
Bad file: 2493379.png
Bad file: 2496963.png
Bad file: 2503933.png
Bad file: 2512047.png
Bad file: 2516421.png
Bad file: 2516571.png
Bad file: 2521978.png
Bad file: 2522389.png
Bad file: 2526619.png
Bad file: 2529175.png
Bad file: 2531553.png
Bad file: 2536168.png
Bad file: 2551833.png
Bad file: 2553655.png
Bad file: 2556630.png
Bad file: 2556770.png
Bad file: 2558982.png
Bad file: 2561534.png
Bad file: 2562550.png
Bad file: 256298.png
Bad file: 256382.png
Bad file: 2565449.png
Bad file: 256782.png
Bad file: 2572979.png
Bad file: 2581164.png
Bad file: 2583986.png
Bad file: 2584162.png
Bad file: 2585330.png
Bad file: 2586168.png
Bad file: 2586511.png
Bad file: 2591085.png
Bad file: 2592477.png
Bad file: 2604562.png
Bad file: 2604784.png
Bad file: 26059

Bad file: 956110.png
Bad file: 962298.png
Bad file: 963135.png
Bad file: 965910.png
Bad file: 970188.png
Bad file: 970482.png
Bad file: 97908.png
Bad file: 981153.png
Bad file: 981415.png
Bad file: 983635.png
Bad file: 984860.png
Bad file: 99360.png
[1006951, 1007762, 1010583, 1012717, 1013445, 1018792, 1022543, 1024948, 1029012, 103089, 1034828, 1035169, 1042929, 1043065, 1045449, 1048066, 1048214, 105067, 1053247, 1055913, 1058284, 106226, 1064937, 1065785, 1096882, 1101031, 1113691, 1115702, 1134195, 1142352, 1142386, 1143249, 1145298, 1157305, 1161021, 1161151, 1161292, 1163998, 1166245, 1170375, 1173401, 1173627, 1175244, 1177796, 1178884, 1187642, 1193658, 1196729, 1203132, 1214279, 1219056, 1224248, 1226395, 122879, 1233545, 1235377, 1236175, 1238872, 1248389, 1248995, 1249897, 1254416, 1254725, 1258460, 1263839, 1271336, 1273476, 1278753, 1283682, 1284864, 1286139, 1289842, 1292832, 1296876, 1299070, 1300790, 130614, 130971, 1313631, 1314615, 1324787, 1329291, 1333240, 134703, 